In [1]:
!pip3 install --upgrade google-cloud-vision
!pip3 install --upgrade google-cloud-translate
!pip3 install --upgrade google-cloud-texttospeech
!pip3 install --upgrade google-cloud-speech
!pip3 install ipywebrtc                             # will auto enable for notebook >= 5.3
!pip3 install gtts
!pip3 install librosa
!pip3 install google-cloud-language
!pip3 install ipywidgets
!pip3 install jupyterlab-widgets

Requirement already up-to-date: google-cloud-vision in /opt/conda/lib/python3.8/site-packages (2.3.0)
Requirement already up-to-date: google-cloud-translate in /opt/conda/lib/python3.8/site-packages (3.1.0)


Requirement already up-to-date: google-cloud-texttospeech in /opt/conda/lib/python3.8/site-packages (2.3.0)
Requirement already up-to-date: google-cloud-speech in /opt/conda/lib/python3.8/site-packages (2.2.1)


In [3]:
import io
import os

# Imports the Google Cloud client libraries
from google.api_core.exceptions import AlreadyExists
from google.cloud import texttospeech
from google.cloud import translate_v3beta1 as translate
from google.cloud import vision
from google.cloud import speech
from google.cloud import language_v1
from gtts import gTTS
from IPython.display import Audio
import IPython.display as ipd
import librosa
import numpy as np

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Test1.json"

def pic_to_text(infile):
    """Detects text in an image file

    ARGS
    infile: path to image file

    RETURNS
    String of text detected in image
    """

    # Instantiates a client
    client = vision.ImageAnnotatorClient()

    # Opens the input image file
    with io.open(infile, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # For dense text, use document_text_detection
    # For less dense text, use text_detection
    response = client.document_text_detection(image=image)
    text = response.full_text_annotation.text
    print("Detected text: {}".format(text))

    return text

def detect_labels(path):
    """Detects labels in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations
    print('Labels:')

    for label in labels:
        print(label.description)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

def sample_analyze_sentiment(text_content):
    """
    Analyzing Sentiment in a String

    Args:
      text_content The text content to analyze
    """

    client = language_v1.LanguageServiceClient()

    # text_content = 'I am so happy and joyful.'

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type_": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8

    response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    # Get overall sentiment of the input document
    print(u"Document sentiment score: {}".format(response.document_sentiment.score))
    print(
        u"Document sentiment magnitude: {}".format(
            response.document_sentiment.magnitude
        )
    )
    if response.document_sentiment.score > 0.8:
        print("This text is strongly positive")
    elif response.document_sentiment.score > 0.4: 
        print("This text is somewhat positive")
    elif response.document_sentiment.score > 0.0: 
        print("This text is slightly positive")
    elif response.document_sentiment.score > -0.4: 
        print("This text is slightly negative")
    elif response.document_sentiment.score > -0.8: 
        print("This text is somewhat negative")
    elif response.document_sentiment.score > -1.1: 
        print("This text is strongly negative ")

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))
    
  
    

        
def detect_document(path):
    """Detects document features in an image."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)

    text = response.full_text_annotation
    
    word_text = ""
    
    #this next block of code parses google cloud's response
    #down to words detected, which are combined into word_text
    for page in text.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    word_text += " "
                    word_text += ''.join([
                        symbol.text for symbol in word.symbols
                        ])    
    print("Handwriting: {}".format(word_text))
    
    return(word_text)
    
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
        

def show_audio_with_controls(file_path):
    display(HTML("<audio controls><source src={} type='audio/mpeg'></audio>".format(file_path)))
    
def main():
    print("Starting Up")
    
    infile = "resources/busy.jpg"    
    handwriting_file = "resources/handwriting.jpg"
    fire_file = "resources/TubbsFire.jpg"
    
    detect_labels(fire_file)
    word_text = detect_document(handwriting_file)

    tts = gTTS(word_text)
    tts.save("resources/output.mp3")
    
    sample_analyze_sentiment("I am so happy")
    
    audio_element_mp3 = ipd.Audio(filename="resources/output.mp3")
    ipd.display(audio_element_mp3)

if __name__ == "__main__":
    # call main
    main()

Starting Up
Labels:
Water resources
World
Urban design
Residential area
Landscape
Geological phenomenon
Water
City
Bird's-eye view
Human settlement
Handwriting:  Happy , Friday BI .
Document sentiment score: 0.8999999761581421
Document sentiment magnitude: 0.8999999761581421
This text is strongly positive
Language of the text: en
